In [1]:
from pydub import AudioSegment
import soundfile as sf
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from IPython.display import Audio, display
import time
import os

# Load the TTS model
config = XttsConfig()
config.load_json("XTTS-v2\\config.json")
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_dir="XTTS-v2", eval=True, vocab_path="XTTS-v2\\vocab.json")
model.cuda()

Xtts(
  (gpt): GPT(
    (conditioning_encoder): ConditioningEncoder(
      (init): Conv1d(80, 1024, kernel_size=(1,), stride=(1,))
      (attn): Sequential(
        (0): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Identity()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (1): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Identity()
          (proj_out): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,))
        )
        (2): AttentionBlock(
          (norm): GroupNorm32(32, 1024, eps=1e-05, affine=True)
          (qkv): Conv1d(1024, 3072, kernel_size=(1,), stride=(1,))
          (attention): QKVAttention()
          (x_proj): Ide

In [1]:
text = """
Hallo, mein Name ist Eren. Und, äh — und ich mag Pizza. Aber ich habe auch andere Interessen, wie zum Beispiel Tic Tac Toe zu spielen. 
Cuando no estoy leyendo, me gusta pasar tiempo al aire libre. Me encanta hacer senderismo en las montañas y explorar los senderos naturales. 
Une autre passion que j'ai est la technologie. Je suis toujours excité d'apprendre les dernières avancées en intelligence artificielle, en robotique et en exploration spatiale. 
Je crois que la technologie a le potentiel de résoudre de nombreux problèmes mondiaux et d'améliorer notre qualité de vie.
In meiner Freizeit spiele ich auch gerne Videospiele. Einige meiner Lieblingsspiele sind Strategiespiele wie Civilization und Echtzeit-Strategiespiele wie StarCraft. 
Ich finde, dass diese Spiele meinen Geist herausfordern und mir helfen, strategisches Denken zu entwickeln. 
Außerdem machen sie viel Spaß, mit Freunden und Familie zu spielen.

"""

In [1]:
import time
import nltk
from nltk.tokenize import sent_tokenize
from pydub import AudioSegment
import soundfile as sf
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

# Download the necessary NLTK data files
nltk.download('punkt')

# Define a list of test texts in different languages and with mixed grammar
test_texts = [
    # Short and simple sentences
    "Hallo.",
    "Ich mag Pizza.",
    "Hola.",
    "Me gusta leer.",
    "Bonjour.",
    "J'aime la technologie.",
    "Ciao.",
    "Mi piace la musica.",
    "こんにちは。",
    "音楽が好きです。",
    "Hello.",
    "I like reading.",

    # Longer and more complex sentences
    "Hallo, mein Name ist Suno. Und, äh — und ich mag Pizza.",
    "Cuando no estoy leyendo, me gusta pasar tiempo al aire libre.",
    "Une autre passion que j'ai est la technologie.",
    "Quando si parla di esplorazione spaziale, sono un grande fan della NASA e di SpaceX.",
    "音楽は私の人生のもう一つの重要な部分です。",
    "Por ejemplo, las nueces de Brasil se llaman 'castañas de Brasil' en español, y son deliciosas.",

    # Very long and complex sentences
    "Hallo, mein Name ist Suno. Ich bin ein großer Fan von Pizza, besonders wenn sie mit frischen Zutaten und viel Käse zubereitet wird. "
    "In meiner Freizeit lese ich gerne Bücher, besonders Science-Fiction- und Fantasy-Romane. "
    "Einer meiner Lieblingsautoren ist Isaac Asimov, und ich liebe besonders seine Foundation-Serie. "
    "Die Art und Weise, wie er komplexe Erzählungen webt und die Zukunft der Menschheit erforscht, ist wirklich faszinierend.",

    "Cuando no estoy leyendo, me gusta pasar tiempo al aire libre. Me encanta hacer senderismo en las montañas y explorar los senderos naturales. "
    "Hay algo en estar rodeado de naturaleza que me ayuda a despejar la mente y encontrar la paz. "
    "También disfruto acampar bajo las estrellas y escuchar los sonidos del bosque por la noche. "
    "Es una excelente manera de desconectar del ajetreo y el bullicio de la vida cotidiana.",

    "Une autre passion que j'ai est la technologie. Je suis toujours excité d'apprendre les dernières avancées en intelligence artificielle, en robotique et en exploration spatiale. "
    "Je crois que la technologie a le potentiel de résoudre de nombreux problèmes mondiaux et d'améliorer notre qualité de vie. "
    "Par exemple, l'IA peut nous aider à prendre de meilleures décisions en analysant de grandes quantités de données, tandis que les robots peuvent aider dans des tâches dangereuses ou répétitives.",

    "Quando si parla di esplorazione spaziale, sono un grande fan della NASA e di SpaceX. L'idea che gli esseri umani possano viaggiare su Marte e oltre è incredibilmente emozionante. "
    "Seguo da vicino le ultime missioni e sviluppi, e spero di vedere una missione umana su Marte nella mia vita. "
    "Il pensiero di esplorare nuovi mondi e possibilmente trovare vita extraterrestre ha sempre alimentato la mia immaginazione.",

    "音楽は私の人生のもう一つの重要な部分です。私はクラシックからロック、電子音楽まで、さまざまなジャンルの音楽を聴くのが好きです。 "
    "音楽には感情を呼び起こし、思い出を作る力があると思います。 "
    "時々、ギターを弾いて自分の曲を作るのも好きです。 "
    "それは自分を表現し、長い一日の後にリラックスする素晴らしい方法です。",

    "Por ejemplo, las nueces de Brasil se llaman 'castañas de Brasil' en español, y son deliciosas. "
    "Me gusta comerlas como un snack saludable. "
    "Además, creo en la importancia de devolver algo a la comunidad. "
    "Trabajo como voluntario en refugios locales y participo en varios eventos benéficos. "
    "Ayudar a los demás y tener un impacto positivo en el mundo es algo que me esfuerzo por lograr. "
    "Creo que incluso pequeños actos de bondad pueden hacer una gran diferencia en la vida de alguien."
]

# Load the TTS model
config = XttsConfig()
config.load_json("XTTS-v2\\config.json")
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_dir="XTTS-v2", eval=True)
model.cuda()

# Function to measure generation time and audio duration for each sentence and character
def measure_generation_time_and_audio_duration(text):
    sentences = sent_tokenize(text)
    total_time = 0
    total_audio_duration = 0
    total_chars = 0
    total_tokens = 0

    for sentence in sentences:
        start_time = time.time()
        outputs = model.synthesize(
            text=sentence,
            config=config,
            speaker_wav="XTTS-v2\\samples\\morgan_freeman.wav",
            gpt_cond_len=30,
            language="de",
            speed=1.0
        )
        end_time = time.time()
        generation_time = end_time - start_time

        # Save the generated audio to a temporary file and measure its duration
        temp_audio_path = "temp_audio.wav"
        sf.write(temp_audio_path, outputs['wav'], config.audio.sample_rate)
        audio = AudioSegment.from_wav(temp_audio_path)
        audio_duration = len(audio) / 1000.0  # Convert from milliseconds to seconds

        total_time += generation_time
        total_audio_duration += audio_duration
        total_chars += len(sentence)
        total_tokens += len(sentence.split())

    avg_time_per_char = total_time / total_chars if total_chars > 0 else 0
    avg_time_per_token = total_time / total_tokens if total_tokens > 0 else 0
    avg_audio_duration_per_char = total_audio_duration / total_chars if total_chars > 0 else 0
    avg_audio_duration_per_token = total_audio_duration / total_tokens if total_tokens > 0 else 0

    return avg_time_per_char, avg_time_per_token, avg_audio_duration_per_char, avg_audio_duration_per_token

# Run the test for each text and calculate the rates
results = []

for text in test_texts:
    avg_time_per_char, avg_time_per_token, avg_audio_duration_per_char, avg_audio_duration_per_token = measure_generation_time_and_audio_duration(text)
    results.append({
        "text": text,
        "avg_time_per_char": avg_time_per_char,
        "avg_time_per_token": avg_time_per_token,
        "avg_audio_duration_per_char": avg_audio_duration_per_char,
        "avg_audio_duration_per_token": avg_audio_duration_per_token
    })

# Print the results and calculate overall averages
total_chars = 0
total_tokens = 0
total_time = 0
total_audio_duration = 0

for result in results:
    print(f"Text: {result['text']}")
    print(f"Average generation time per character: {result['avg_time_per_char']} seconds")
    print(f"Average generation time per token: {result['avg_time_per_token']} seconds")
    print(f"Average audio duration per character: {result['avg_audio_duration_per_char']} seconds")
    print(f"Average audio duration per token: {result['avg_audio_duration_per_token']} seconds")
    print()
    
    total_chars += len(result['text'])
    total_tokens += len(result['text'].split())
    total_time += result['avg_time_per_char'] * len(result['text'])
    total_audio_duration += result['avg_audio_duration_per_char'] * len(result['text'])

# Calculate overall averages
overall_avg_time_per_char = total_time / total_chars if total_chars > 0 else 0
overall_avg_time_per_token = total_time / total_tokens if total_tokens > 0 else 0
overall_avg_audio_duration_per_char = total_audio_duration / total_chars if total_chars > 0 else 0
overall_avg_audio_duration_per_token = total_audio_duration / total_tokens if total_tokens > 0 else 0

print("Overall Averages:")
print(f"Overall average generation time per character: {overall_avg_time_per_char} seconds")
print(f"Overall average generation time per token: {overall_avg_time_per_token} seconds")
print(f"Overall average audio duration per character: {overall_avg_audio_duration_per_char} seconds")
print(f"Overall average audio duration per token: {overall_avg_audio_duration_per_token} seconds")

# Compare texts to each other
print("\nComparison of Texts:")
sorted_results = sorted(results, key=lambda x: x['avg_time_per_char'])
for i, result in enumerate(sorted_results):
    print(f"{i+1}. Text: {result['text'][:50]}...")  # Print the first 50 characters for brevity
    print(f"   Average generation time per character: {result['avg_time_per_char']} seconds")
    print(f"   Average generation time per token: {result['avg_time_per_token']} seconds")
    print(f"   Average audio duration per character: {result['avg_audio_duration_per_char']} seconds")
    print(f"   Average audio duration per token: {result['avg_audio_duration_per_token']} seconds")
    print()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kalin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\kalin\anaconda3\envs\torch\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


Text: Hallo.
Average generation time per character: 2.2236289580663047 seconds
Average generation time per token: 13.341773748397827 seconds
Average audio duration per character: 2.291 seconds
Average audio duration per token: 13.746 seconds

Text: Ich mag Pizza.
Average generation time per character: 0.13582447596958705 seconds
Average generation time per token: 0.6338475545247396 seconds
Average audio duration per character: 0.1617142857142857 seconds
Average audio duration per token: 0.7546666666666666 seconds

Text: Hola.
Average generation time per character: 0.4366607189178467 seconds
Average generation time per token: 2.1833035945892334 seconds
Average audio duration per character: 0.5456000000000001 seconds
Average audio duration per token: 2.728 seconds

Text: Me gusta leer.
Average generation time per character: 0.19866129330226354 seconds
Average generation time per token: 0.9270860354105631 seconds
Average audio duration per character: 0.25207142857142856 seconds
Average au

In [1]:
import time
from pydub import AudioSegment
import soundfile as sf
import threading
from pydub.playback import play
import os
import gc
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
import torch

class TTSStreamer:
    def __init__(self, model_path, config_path, vocab_path, speaker_wav="XTTS-v2\\samples\\morgan_freeman.wav"):
        self.model_path = model_path
        self.config_path = config_path
        self.vocab_path = vocab_path
        self.speaker_wav = speaker_wav
        self.model = self.load_model()

    def load_model(self):
        config = XttsConfig()
        config.load_json(self.config_path)
        model = Xtts.init_from_config(config)
        model.load_checkpoint(config, checkpoint_dir=self.model_path, eval=True, vocab_path=self.vocab_path)
        model.cuda()
        return model


    def unload_model(self):
        del self.model
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gc.collect()
        print("Model unloaded and GPU memory cleared successfully.")

    def estimate_times(self, text_chunk, avg_gen_time_per_char, avg_audio_time_per_char):
        gen_time = len(text_chunk) * avg_gen_time_per_char
        audio_duration = len(text_chunk) * avg_audio_time_per_char
        return gen_time, audio_duration

    def split_text_into_lines(self, text):
        lines = text.strip().split('\n')
        return lines

    def generate_audio_chunk(self, chunk, chunk_index, audio_buffer, playback_event, avg_gen_time_per_char, avg_audio_time_per_char, total_gen_time, language, speed):
        est_gen_time, est_audio_duration = self.estimate_times(chunk, avg_gen_time_per_char, avg_audio_time_per_char)
        print(f"Chunk {chunk_index + 1} estimated generation time: {est_gen_time:.2f} seconds, estimated audio duration: {est_audio_duration:.2f} seconds")

        print(f"Generating audio for chunk {chunk_index + 1}...")
        start_gen_time = time.time()
        outputs = self.model.synthesize(
            text=chunk,
            config=self.model.config,
            speaker_wav=self.speaker_wav,
            gpt_cond_len=30,
            language=language,
            speed=speed
        )
        end_gen_time = time.time()
        generation_time = end_gen_time - start_gen_time
        total_gen_time[0] += generation_time
        print(f"Chunk {chunk_index + 1} generated in {generation_time:.2f} seconds (estimated: {est_gen_time:.2f} seconds)")

        wav_data = outputs['wav']
        temp_output_file = f'temp_output_{chunk_index}.wav'
        sf.write(temp_output_file, wav_data, 22050)
        line_audio = AudioSegment.from_wav(temp_output_file)

        actual_audio_duration = len(line_audio) / 1000.0
        print(f"Chunk {chunk_index + 1} actual audio duration: {actual_audio_duration:.2f} seconds (estimated: {est_audio_duration:.2f} seconds)")

        audio_buffer[chunk_index] = line_audio
        print(f"Chunk {chunk_index + 1} audio saved and buffered")

        playback_event.set()

    def stream_audio_with_buffering(self, text, language="en", speed=1.2, speaker_path=None, fireup_delay=1.0, avg_gen_time_per_char=0.08058659382140704, avg_audio_time_per_char=0.1064346054068992):
        if speaker_path:
            self.speaker_wav = speaker_path

        print("Starting the audio streaming process...")
        start_time = time.time()

        text_chunks = self.split_text_into_lines(text)
        audio_buffer = [None] * len(text_chunks)
        playback_events = [threading.Event() for _ in text_chunks]
        total_gen_time = [0]

        def start_playback_after_delay():
            print(f"Waiting {fireup_delay:.2f} seconds before starting playback...")
            time.sleep(fireup_delay)
            print("Fireup delay is over, starting playback...")
            for chunk_index in range(len(text_chunks)):
                playback_events[chunk_index].wait()
                if audio_buffer[chunk_index] is not None:
                    self.play_audio_segment(audio_buffer[chunk_index])

        playback_thread = threading.Thread(target=start_playback_after_delay)
        playback_thread.start()

        for chunk_index, chunk in enumerate(text_chunks):
            print(f"Processing chunk {chunk_index + 1}/{len(text_chunks)}: '{chunk}'")
            self.generate_audio_chunk(chunk, chunk_index, audio_buffer, playback_events[chunk_index], avg_gen_time_per_char, avg_audio_time_per_char, total_gen_time, language, speed)

        playback_thread.join()
        print("Audio streaming process completed.")
        print(f"Total generation time: {total_gen_time[0]:.2f} seconds")

    def play_audio_segment(self, audio_segment):
        play(audio_segment)


# Initialize the TTSStreamer
tts_streamer = TTSStreamer(model_path="XTTS-v2", config_path="XTTS-v2\\config.json", vocab_path="XTTS-v2\\vocab.json")

In [2]:
# Example usage
text = """
A series of extracts from Middle English texts of different dates and types is presented, with commentary on all of the loanwords 
(from French, Latin, or early Scandinavian) found in each passage. 
The extracts are drawn from the Final Continuation of the Peterborough Chronicle; 
the Ormulum; the Ancrene Wisse; John Trevisa’s translation of Ranulf Higden’s Polychronicon; and Caxton’s Prologue to The Boke of Eneydos. 
This is complemented by a more discursive consideration of some passages from multilingual texts and texts not in English, 
drawing attention to the questions they raise about some of the likely mechanisms by which words entered English, 
as well about how far individuals distinguished between the vocabularies of each language in a situation of functional language switching. 
The chapter is followed by a brief summary of the main conclusions from chapters 11, 12, and 13.
"""
# Stream audio with buffering
tts_streamer.stream_audio_with_buffering(text, language="en", speed=1.3, speaker_path="XTTS-v2\samples\Eren_test.wav", fireup_delay=5.0)

Starting the audio streaming process...
Waiting 5.00 seconds before starting playback...
Processing chunk 1/8: 'A series of extracts from Middle English texts of different dates and types is presented, with commentary on all of the loanwords '
Chunk 1 estimated generation time: 10.48 seconds, estimated audio duration: 13.84 seconds
Generating audio for chunk 1...
Fireup delay is over, starting playback...
Chunk 1 generated in 7.79 seconds (estimated: 10.48 seconds)
Chunk 1 actual audio duration: 7.02 seconds (estimated: 13.84 seconds)
Chunk 1 audio saved and buffered
Processing chunk 2/8: '(from French, Latin, or early Scandinavian) found in each passage. '
Chunk 2 estimated generation time: 5.40 seconds, estimated audio duration: 7.13 seconds
Generating audio for chunk 2...
Chunk 2 generated in 3.68 seconds (estimated: 5.40 seconds)
Chunk 2 actual audio duration: 3.79 seconds (estimated: 7.13 seconds)
Chunk 2 audio saved and buffered
Processing chunk 3/8: 'The extracts are drawn from 

In [3]:
text = """
1. Was genau macht MultilingualPress?
MultilingualPress stellt Sprachbeziehungen zwischen verschiedenen Sites einer WordPress Multisite her. 
Somit ermöglicht es dir, deinen Websitebesuchern Inhalte in der für sie passenden Sprachversion anzuzeigen. 
Dabei legst du jede Sprachversion deiner Website als Site in einer WordPress Multisite an und verknüpfst die übersetzten Inhalte mithilfe von MultilingualPress. 
Inhalte sind beispielsweise Beiträge, Seiten, Custom Post Types oder auch sogenannte Taxonomien. Zu letzterem gehören zum Beispiel Kategorien und Schlagworte.

Unser Plugin hilft dir darüberhinaus dabei, die Website-Besucher auf die passende Sprachversion weiterzuleiten. 
Es unterstützt dich automatisch auch bei der Suchmaschinenoptimierung deiner mehrsprachigen Website.
2. Sieh dir unser MultilingualPress Video an 
Du schaust dir lieber ein Video an als einen langen Text zu lesen? Du möchtest wissen wie MultilingualPress im Backend aussieht und dich von der leichten Bedienbarkeit überzeugen, 
bevor du das Produkt kaufst? Dann sieh dir hier unser Produktvideo an.

Sollte das Video nicht alle deine Fragen beantwortet haben, dann lies bitte hier weiter oder melde dich bei uns.
"""
tts_streamer.stream_audio_with_buffering(text, language="de", speed=1.3, speaker_path="XTTS-v2\samples\Eren_test.wav", fireup_delay=5.0)

Starting the audio streaming process...
Waiting 5.00 seconds before starting playback...
Processing chunk 1/13: '1. Was genau macht MultilingualPress?'
Chunk 1 estimated generation time: 2.98 seconds, estimated audio duration: 3.94 seconds
Generating audio for chunk 1...
Chunk 1 generated in 3.34 seconds (estimated: 2.98 seconds)
Chunk 1 actual audio duration: 3.33 seconds (estimated: 3.94 seconds)
Chunk 1 audio saved and buffered
Processing chunk 2/13: 'MultilingualPress stellt Sprachbeziehungen zwischen verschiedenen Sites einer WordPress Multisite her. '
Chunk 2 estimated generation time: 8.30 seconds, estimated audio duration: 10.96 seconds
Generating audio for chunk 2...
Fireup delay is over, starting playback...
Chunk 2 generated in 6.82 seconds (estimated: 8.30 seconds)
Chunk 2 actual audio duration: 7.22 seconds (estimated: 10.96 seconds)
Chunk 2 audio saved and buffered
Processing chunk 3/13: 'Somit ermöglicht es dir, deinen Websitebesuchern Inhalte in der für sie passenden Sp

KeyboardInterrupt: 

In [9]:
from pydub import AudioSegment
from moviepy.editor import VideoFileClip

def convert_and_clip_audio(input_file, output_file, target_sample_rate=24000, clip_duration=15):
    # Determine the file format
    file_format = input_file.split('.')[-1].lower()
    
    if file_format == 'mp4':
        # Load the MP4 file
        video = VideoFileClip(input_file)
        audio = video.audio
        audio.write_audiofile("temp_audio.wav")
        audio = AudioSegment.from_file("temp_audio.wav")
    else:
        # Load the input file (assumed to be WAV)
        audio = AudioSegment.from_file(input_file)
    
    # Convert to mono
    audio = audio.set_channels(1)
    
    # Resample to the target sample rate
    audio = audio.set_frame_rate(target_sample_rate)
    
    # Calculate the start and end times for clipping
    start_time = (len(audio) - clip_duration * 1000) // 2
    end_time = start_time + clip_duration * 1000
    
    # Clip the audio
    clipped_audio = audio[start_time:end_time]
    
    # Save the output file
    clipped_audio.export(output_file, format="wav")

# Paths to the audio files
input_file = r"2024-08-15 21-48-51 (enhanced).wav"  # or .wav
output_file = "XTTS-v2\samples\Eren_test.wav"

# Convert and clip the audio file
convert_and_clip_audio(input_file, output_file)

In [3]:
import threading
import queue
from pydub import AudioSegment
from pydub.playback import play
import soundfile as sf
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
import time
import os

# Function to synthesize audio for a line of text
def synthesize_audio(line, config, model, index, audio_queue):
    outputs = model.synthesize(
        text=line,
        config=config,
        speaker_wav="XTTS-v2\\samples\\morgan_freeman.wav",
        gpt_cond_len=30,
        language="en",
        speed=1.1
    )
    wav_data = outputs['wav']
    temp_output_file = f'temp_output_{index}.wav'
    sf.write(temp_output_file, wav_data, 22050)  # Assuming the sample rate is 22050 Hz
    audio_queue.put(temp_output_file)

# Function to play audio from the queue
def play_audio(audio_queue):
    while True:
        audio_file = audio_queue.get()
        if audio_file is None:
            break
        audio_segment = AudioSegment.from_wav(audio_file)
        play(audio_segment)
        os.remove(audio_file)  # Clean up the temporary file
        audio_queue.task_done()

# Split the text into lines
lines = text.strip().split('\n')

# Load the TTS model
config = XttsConfig()
config.load_json("XTTS-v2\\config.json")
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_dir="XTTS-v2", eval=True)
model.cuda()

start_time = time.time()

# Create a queue to hold the audio files
audio_queue = queue.Queue()

# Start the audio playback thread
playback_thread = threading.Thread(target=play_audio, args=(audio_queue,))
playback_thread.start()

# Synthesize audio for each line in separate threads
threads = []
for index, line in enumerate(lines):
    thread = threading.Thread(target=synthesize_audio, args=(line, config, model, index, audio_queue))
    threads.append(thread)
    thread.start()

# Wait for all synthesis threads to finish
for thread in threads:
    thread.join()

# Signal the playback thread to stop
audio_queue.put(None)
playback_thread.join()

end_time = time.time()
generation_time = end_time - start_time

# Print the generation time
print(f"Total generation time: {generation_time} seconds")

Exception in thread Thread-8 (synthesize_audio):
Traceback (most recent call last):
  File "c:\Users\kalin\anaconda3\envs\torch\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\kalin\anaconda3\envs\torch\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
Exception in thread Thread-6 (synthesize_audio):
Traceback (most recent call last):
  File "c:\Users\kalin\anaconda3\envs\torch\lib\threading.py", line 1016, in _bootstrap_inner
    _threading_Thread_run(self)
  File "c:\Users\kalin\anaconda3\envs\torch\lib\threading.py", line 953, in run
    self.run()
  File "c:\Users\kalin\anaconda3\envs\torch\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    self._target(*self._args, **self._kwargs)
  File "C:\Users\kalin\AppData\Local\Temp\ipykernel_10492\1680889571.py", line 13, in synthesize_audio
    _threading_Thread_run(self)
  File "c:\Users\kalin\anaconda3\envs\torch\lib\threading.py", line 953, in run
    self._target(*s

Total generation time: 1.1605887413024902 seconds


In [8]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa
import torch

# Load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.config.forced_decoder_ids = None

# Load the MP3 file
file_path = "output_combined - Copy.mp3"
audio_array, original_sampling_rate = librosa.load(file_path, sr=None)

# Resample the audio to 16000 Hz
target_sampling_rate = 16000
if original_sampling_rate != target_sampling_rate:
    audio_array = librosa.resample(audio_array, orig_sr=original_sampling_rate, target_sr=target_sampling_rate)

# Process the audio file to extract input features
input_features = processor(audio_array, sampling_rate=target_sampling_rate, return_tensors="pt").input_features

# Generate token IDs
predicted_ids = model.generate(input_features)

# Decode token IDs to text
transcription_with_special_tokens = processor.batch_decode(predicted_ids, skip_special_tokens=False)
transcription_without_special_tokens = processor.batch_decode(predicted_ids, skip_special_tokens=True)

print(transcription_without_special_tokens)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[" Hello. My name is Aaron Kalinsatsilio-Glu. I build meaningful experiences. Let's chat. Besides coding, I enjoy gaming and cooking. Recently, I've been delving into machine learning to make a lasting impact. Alongside my expertise in 3D modeling, animation, software development, and AI, I possess strong leadership and problem-solving skills, refined through innovative projects and AI-driven educational tools development. My leadership, problem-solving,"]


In [2]:
import whisper

model = whisper.load_model("small")
result = model.transcribe("output_combined.wav")
print(result["text"])

 Hallo, mein Name ist Ehrn und Ehe und ich mag Pizza. Aber ich habe auch andere Interessen wie zum Beispiel Tic Tac To zu spielen. Wann du nicht leidest, magst du die Zeit für die Ehe frei. Ich liebe den Senderismus, in den Mountains und den naturalen Senderes zu verspielen. Eine andere Passion, die ich in der Technologie habe. Ich bin immer exakt am letzten Abenteuer in die Art und Weise. In der Robotik und in der Exploration. Je quoi, wie die Technologie, alle Potentialen, die Resorten, die Nombro, Problemen, Mondio und der Melioré, unsere Qualität. In meiner Freizeitspiele, ich auch gerne Videospiele. Einige meiner Lieblingsspiele sind Strategiespiele wie Zivilisation und Echtzeit-Strategiespiele wie Starcraft. Ich finde, dass diese Spiele meinen Geist herausfordern und mir helfen, strategisches Denken zu entwickeln. Außerdem machen sie viel Spaß, mit Freunden und Familie, zu spielen.
